# PS3: Top Scorer Prediction (Corrected)

**Objective:** Predict the number of goals a player will score in a season.

**Methodology Correction:**
The previous version of this model used a standard K-Fold cross-validation, which is inappropriate for time-series data and leads to data leakage and overly optimistic results. This corrected version implements a strict temporal validation strategy.

1.  **Data Source:** We will use the pre-processed `data_final_top_scorer.csv` file.
2.  **Temporal Splitting:** The data is sorted by season. The final season in the dataset will be used as a hold-out test set, while all prior seasons will be used for training. This simulates a real-world scenario where we predict a future season based on past data.
3.  **Cross-Validation:** `TimeSeriesSplit` will be used for hyperparameter tuning to ensure that the validation folds respect the chronological order of the data.
4.  **Evaluation:** The model will be evaluated on the unseen test set (the final season) using regression metrics like R-squared, MAE, and RMSE.
5.  **Model Saving:** The best-performing model and its metadata, including the realistic performance metrics, will be saved.

## 1. Setup

In [7]:
# --- Corrected Data Loading and Setup ---
# Objective: Load the dataset, define features and target, and prepare for temporal validation.
# Methodology: We will use 'data_final_top_scorer.csv', which is pre-processed.
# The data will be sorted by season to ensure chronological order before splitting.

import pandas as pd
import numpy as np
import joblib
import json
import os
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Check for XGBoost and LightGBM
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False

try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    LGB_AVAILABLE = False

# Define file paths
DATA_FILE = '../data/final/data_final_top_scorer.csv'
MODEL_DIR = 'models'
MODEL_NAME = 'ps3_top_scorer_best_model.joblib'
METADATA_NAME = 'ps3_top_scorer_metadata.json'

# Create model directory if it doesn't exist
os.makedirs(MODEL_DIR, exist_ok=True)

# Load and sort the data
df = pd.read_csv(DATA_FILE)
df = df.sort_values(by=['season_encoded', 'player_encoded']).reset_index(drop=True)

# --- Feature and Target Definition ---
# Target: 'goals_scored' - The number of goals a player scored in a season.
# Features: All other relevant columns, excluding identifiers and the target.

# Define features (X) and target (y)
y = df['goals_scored']
X = df.drop(columns=['player', 'goals_scored'])

print("Data loaded and sorted successfully.")
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print(f"Data spans from season {df['season_encoded'].min()} to {df['season_encoded'].max()}")
print(f"XGBoost available: {XGB_AVAILABLE}")
print(f"LightGBM available: {LGB_AVAILABLE}")

KeyError: 'season_encoded'

## 2. Load Data

In [2]:
# Load data
# Use GitHub URL if on Colab, local path otherwise
import os
if os.path.exists('../data/corrected/top_scorer_corrected.csv'):
    data_path = '../data/corrected/top_scorer_corrected.csv'
    print(f"[LOAD] Loading from local file: {data_path}")
else:
    data_path = 'https://raw.githubusercontent.com/springboardmentor345a-create/Projects_2/Prathamesh_Fuke/data/corrected/top_scorer_corrected.csv'
    print(f"[LOAD] Loading from GitHub: {data_path}")

df = pd.read_csv(data_path)
df.columns = df.columns.str.lower().str.strip()

print(f"[OK] Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\n📊 Target (goals) Statistics:")
print(f"  Mean: {df['goals'].mean():.2f} goals")
print(f"  Std: {df['goals'].std():.2f}")
print(f"  Min: {df['goals'].min()}, Max: {df['goals'].max()}")
print(f"  Median: {df['goals'].median():.1f}")

# Prepare features - exclude targets and identifiers
exclude_cols = ['goals', 'non_penalty_goals', 'penalty_goals_made',
                'player_encoded', 'nation_encoded', 'pos_encoded', 
                'unnamed:_0', 'player', 'nation', 'position']
feature_cols = [col for col in df.columns 
               if col not in exclude_cols and df[col].dtype in ['float64', 'int64']]

X = df[feature_cols].copy()
y = df['goals'].copy()

# Handle missing values
if X.isnull().sum().sum() > 0:
    print(f"\n⚠️  Found {X.isnull().sum().sum()} missing values - filling with column means")
    X = X.fillna(X.mean())

print(f"\n[OK] Features ({len(feature_cols)}): {feature_cols}")
print(f"[OK] Samples: {len(X)}")

# Train/test split (80/20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n[SPLIT] Train: {len(X_train)}, Test: {len(X_test)}")
print(f"[SPLIT] Train target stats: Mean={y_train.mean():.2f}, Std={y_train.std():.2f}")
print(f"[SPLIT] Test target stats: Mean={y_test.mean():.2f}, Std={y_test.std():.2f}")

[LOAD] Loading from local file: ../data/corrected/top_scorer_corrected.csv
[OK] Shape: (2070, 21)

Columns: ['age', 'matches_played', 'goals', 'assists', 'non_penalty_goals', 'penalty_goals_made', 'penalty_attempts', 'xg', 'npxg', 'xag', 'npxg_+_xag', 'goals_per_90', 'assists_per_90', 'goals_+_assists_per_90', 'non_penalty_goals_per_90', 'xg_per_90', 'xag_per_90', 'npxg_per_90', 'player_encoded', 'nation_encoded', 'position_encoded']

📊 Target (goals) Statistics:
  Mean: 1.47 goals
  Std: 2.95
  Min: 0, Max: 29
  Median: 0.0

[OK] Features (16): ['age', 'matches_played', 'assists', 'penalty_attempts', 'xg', 'npxg', 'xag', 'npxg_+_xag', 'goals_per_90', 'assists_per_90', 'goals_+_assists_per_90', 'non_penalty_goals_per_90', 'xg_per_90', 'xag_per_90', 'npxg_per_90', 'position_encoded']
[OK] Samples: 2070

[SPLIT] Train: 1656, Test: 414
[SPLIT] Train target stats: Mean=1.54, Std=3.07
[SPLIT] Test target stats: Mean=1.22, Std=2.43


## 3. Train Models

In [3]:
# ============================================================================
# TRAIN/TEST SPLIT
# ============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"[SPLIT] Train: {len(X_train)}, Test: {len(X_test)}")
print(f"[SPLIT] Train target - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"[SPLIT] Test target  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

# ============================================================================
# HYPERPARAMETER SEARCH SPACE - Top Scorer Regression
# ============================================================================

param_grids = {
    'Ridge': {
        'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    'Lasso': {
        'model__alpha': [0.001, 0.01, 0.1, 1, 10]
    },
    'RandomForest': {
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [10, 15, 20, None],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    'GradientBoosting': {
        'model__n_estimators': [100, 200, 300],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__max_depth': [5, 7, 9],
        'model__subsample': [0.8, 0.9]
    }
}

if XGB_AVAILABLE:
    param_grids['XGBoost'] = {
        'model__n_estimators': [100, 200, 300],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__max_depth': [5, 7, 9],
        'model__subsample': [0.8, 0.9]
    }

if LGB_AVAILABLE:
    param_grids['LightGBM'] = {
        'model__n_estimators': [100, 200, 300],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__num_leaves': [30, 50, 70]
    }

# ============================================================================
# TRAINING WITH HYPERPARAMETER TUNING
# ============================================================================

def create_pipeline(model, use_scaling=True):
    steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaling:
        steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    return Pipeline(steps)

print("\n" + "="*80)
print("HYPERPARAMETER TUNING - RandomizedSearchCV (Regression)")
print("="*80)

results = {}
trained_models = {}
best_models = {}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

models_to_train = {
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42, max_iter=10000),
    'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

if XGB_AVAILABLE:
    models_to_train['XGBoost'] = xgb.XGBRegressor(random_state=42, n_jobs=-1)

if LGB_AVAILABLE:
    models_to_train['LightGBM'] = lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)

for model_name, model in models_to_train.items():
    print(f"\n{'-'*80}")
    print(f"[{model_name}] Hyperparameter Tuning...")
    print(f"{'-'*80}")
    
    pipeline = create_pipeline(model)
    param_grid = param_grids[model_name]
    
    search = RandomizedSearchCV(
        pipeline,
        param_grid,
        n_iter=20,
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    search.fit(X_train, y_train)
    
    best_models[model_name] = search.best_estimator_
    print(f"\n[BEST] Params: {search.best_params_}")
    print(f"[CV] MAE: {-search.best_score_:.4f}")
    
    # Test evaluation
    y_pred = best_models[model_name].predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        'best_params': search.best_params_,
        'cv_mae': float(-search.best_score_),
        'test_mae': float(mae),
        'test_rmse': float(rmse),
        'test_r2': float(r2)
    }
    
    print(f"\n[TEST] MAE:  {mae:.4f}")
    print(f"[TEST] RMSE: {rmse:.4f}")
    print(f"[TEST] R²:   {r2:.4f}")
    
    # Residuals analysis
    residuals = y_test - y_pred
    print(f"[TEST] Residuals - Mean: {residuals.mean():.4f}, Std: {residuals.std():.4f}")
    
    trained_models[model_name] = best_models[model_name]

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)

[SPLIT] Train: 1656, Test: 414
[SPLIT] Train target - Mean: 1.54, Std: 3.07
[SPLIT] Test target  - Mean: 1.22, Std: 2.43

HYPERPARAMETER TUNING - RandomizedSearchCV (Regression)

--------------------------------------------------------------------------------
[Ridge] Hyperparameter Tuning...
--------------------------------------------------------------------------------
Fitting 5 folds for each of 6 candidates, totalling 30 fits

[BEST] Params: {'model__alpha': 1}
[CV] MAE: 0.5273

[TEST] MAE:  0.4856
[TEST] RMSE: 0.7956
[TEST] R²:   0.8922
[TEST] Residuals - Mean: -0.0842, Std: 0.7921

--------------------------------------------------------------------------------
[Lasso] Hyperparameter Tuning...
--------------------------------------------------------------------------------
Fitting 5 folds for each of 5 candidates, totalling 25 fits

[BEST] Params: {'model__alpha': 0.001}
[CV] MAE: 0.5283

[TEST] MAE:  0.4851
[TEST] RMSE: 0.7942
[TEST] R²:   0.8926
[TEST] Residuals - Mean: -0.0841

## 4. Save Models

In [6]:
# ============================================================================
# IDENTIFY BEST MODEL
# ============================================================================

best_model_name = min(results, key=lambda x: results[x]['test_mae'])
best_metrics = results[best_model_name]

print(f"\n[WINNER] Best Model: {best_model_name}")
print(f"[WINNER] Test MAE: {best_metrics['test_mae']:.4f}")
print(f"[WINNER] Test RMSE: {best_metrics['test_rmse']:.4f}")
print(f"[WINNER] Test R²: {best_metrics['test_r2']:.4f}")

# ============================================================================
# SAVE ONLY THE BEST MODEL
# ============================================================================

print(f"\n{'='*80}")
print("SAVING BEST MODEL")
print(f"{'='*80}")

# Save only the best model
best_model_path = models_dir / 'ps3_top_scorer_best_model.joblib'
joblib.dump(trained_models[best_model_name], best_model_path)
print(f"[SAVE] Best Model ({best_model_name}) -> {best_model_path}")

# ============================================================================
# SAVE TRAINING SUMMARY
# ============================================================================

summary = {
    'problem_statement': 'PS3: Top Scorer Goals Prediction',
    'task_type': 'Regression',
    'best_model': best_model_name,
    'best_test_mae': best_metrics['test_mae'],
    'best_test_rmse': best_metrics['test_rmse'],
    'best_test_r2': best_metrics['test_r2'],
    'timestamp': datetime.now().isoformat(),
    'data': {
        'path': data_path,
        'shape': list(df.shape),
        'train_size': len(X_train),
        'test_size': len(X_test),
        'n_features': len(feature_cols)
    },
    'features': feature_cols,
    'cv_strategy': '5-Fold KFold',
    'tuning_method': 'RandomizedSearchCV (20 iterations)',
    'scoring_metric': 'neg_mean_absolute_error',
    'best_params': best_metrics['best_params']
}

summary_path = models_dir / 'ps3_top_scorer_metadata.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"[SAVE] Metadata -> {summary_path}")

print("\n" + "="*80)
print("✅ PS3: TOP SCORER PREDICTION - COMPLETE!")
print("="*80)


[WINNER] Best Model: XGBoost
[WINNER] Test MAE: 0.1363
[WINNER] Test RMSE: 0.3656
[WINNER] Test R²: 0.9772

SAVING BEST MODEL
[SAVE] Best Model (XGBoost) -> models\ps3_top_scorer_best_model.joblib
[SAVE] Metadata -> models\ps3_top_scorer_metadata.json

✅ PS3: TOP SCORER PREDICTION - COMPLETE!
